<a href="https://colab.research.google.com/github/saisrikanthrayavarapu/huggingface/blob/master/Causal_analysis_using_Quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the necessary packages

In [ ]:
!pip install lingam -q
!pip install python-igraph -q
!pip install factor_analyzer -q
!pip install qiskit -q
!pip install qiskit-aqua -q
!pip install qiskit-terra -q
!pip install cirq -q
!pip install git+https://github.com/qulacs/cirq-qulacs -q
!pip install Qulacs -q
!pip install asyncpgsa -q
!pip install setuptools -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import functools
import pandas as pd
import graphviz
import itertools
import math
import random
import re
import collections

import lingam
from lingam.utils import make_dot

from sklearn.preprocessing import StandardScaler, MinMaxScaler, KernelCenterer
from sklearn import preprocessing

from qiskit import BasicAer
from qiskit.utils import QuantumInstance
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)
from qiskit import *
from qiskit.tools.visualization import *
from qiskit.circuit.library import *
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator, UnitarySimulator
from qiskit import IBMQ, QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, Aer
from qiskit.qasm import pi
from qiskit.tools.visualization import plot_histogram, circuit_drawer

from typing import cast, Dict, Iterator, List, Union
from cirq import *
import cirq
from cirq import circuits, ops, protocols, linalg
from cirq.sim import density_matrix_utils
from cirq import DensityMatrixSimulator
from cirq import DensityMatrixStepResult
from cirqqulacs import QulacsSimulator
import qulacs
from cirq_google import *
from cirq.contrib.svg import SVGCircuit
from cirq import GridQubit

# Accessing available devices in IBMQ

In [ ]:
TOKEN = '7a3519955cdbb5a9ee5d963e0eef41d5687012c21e19c0f61c8f339579cbb3eb5a990c4971b9edb17b72d81bfbc26cfd66c7072026cf49bb85173a7955af084a'
IBMQ.save_account(TOKEN)
IBMQ.load_account() 
IBMQ.providers()    
ibmq_provider = IBMQ.get_provider()
backend_sim = ibmq_provider.backend.ibmq_qasm_simulator # Select the device to use

configrc.store_credentials:WARNING:2022-08-05 06:25:33,522: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2022-08-05 06:25:33,673: Credentials are already in use. The existing account in the session will be replaced.


# List of Functions

In [ ]:
"""tanh-shrink function"""
def tanh(x):
    return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
def func_one(x):
    return x - tanh(x)
def func_two(x1, x2):
    return (x1 - tanh(x1))*(x2 - tanh(x2))

In [ ]:
from cirq.ops.common_gates import HPowGate
"""quantum kernel using cirq and qulacs"""
#ring connection
def quantum_kernel_cirq_ring(bn, n_samples, data_x, depth):
    
    result_gram_mat = np.eye(n_samples)
    
    pair_list = []
    for pair in itertools.combinations(list(range(n_samples)), 2):
        pair_list.append(list(pair))
    
    #scaling and standardization
    x_standardize = preprocessing.scale(data_x)
    x_input = np.array([2*x_standardize for i in range(bn)])
    
    #calculations for Gram matrix
    for j in range(len(pair_list)):
        q = [GridQubit(i, 0) for i in range(bn)]
        
        #IQP circuit
        def IQP_circuit(qci, bn, input_data, rep):
            for _ in range(rep):
              # Hadamard gate
                qci.append(HPowGate.on_each(q))
                for i in range(bn):
                    qci.append(cirq.ZPowGate(exponent = func_one(input_data[i]))(q[i]))
                for i in range(bn):
                    if i == (bn-1):
                        qci.append(cirq.CZPowGate(exponent = func_two(input_data[bn-1], input_data[0]))(q[bn-1], q[0]))
                    else:
                        qci.append(cirq.CZPowGate(exponent = func_two(input_data[i], input_data[i+1]))(q[i], q[i+1]))
        
        #First half of the circuit
        qc_cirq = cirq.Circuit()
        IQP_circuit(qci = qc_cirq, bn = bn, input_data = x_input[:, pair_list[j][0]], rep = depth)
        result_1 = QulacsSimulator().simulate(qc_cirq)
        #Second half of the circuit
        qcd_cirq = cirq.Circuit()
        IQP_circuit(qci = qcd_cirq, bn = bn, input_data = x_input[:, pair_list[j][1]], rep = depth)
        result_2 = QulacsSimulator().simulate(qcd_cirq)
        #Calculations for Gram matrix using inner product
        state_1 = result_1._final_simulator_state.reshape(1,-1)
        state_2 = result_2._final_simulator_state.reshape(1,-1)
        result_kernel = abs((np.conjugate(state_1) @ state_2.T)[0][0])**2
        result_gram_mat[pair_list[j][0], pair_list[j][1]] = result_kernel
        
    kernel_gram_mat = result_gram_mat + result_gram_mat.T - np.diag(result_gram_mat.diagonal())
    return kernel_gram_mat

In [ ]:
"""quantum kernel using cirq and qulacs"""
#linear connection
def quantum_kernel_cirq_linear(bn, n_samples, data_x, depth):
    
    result_gram_mat = np.eye(n_samples)
    
    pair_list = []
    for pair in itertools.combinations(list(range(n_samples)), 2):
        pair_list.append(list(pair))
    
    #scaling and standardization
    x_standardize = preprocessing.scale(data_x)
    x_input = np.array([2*x_standardize for i in range(bn)])
    
    #calculations for Gram matrix
    for j in range(len(pair_list)):
        q = [GridQubit(i, 0) for i in range(bn)]
        
        #IQP circuit
        def IQP_circuit(qci, bn, input_data, rep):
            for _ in range(rep):
                qci.append(HPowGate.on_each(q))
                for i in range(bn):
                    qci.append(cirq.ZPowGate(exponent = func_one(input_data[i]))(q[i]))
                for i in range(bn-1):
                    qci.append(cirq.CZPowGate(exponent = func_two(input_data[i], input_data[i+1]))(q[i], q[i+1]))
        
        #First half of the circuit
        qc_cirq = cirq.Circuit()
        IQP_circuit(qci = qc_cirq, bn = bn, input_data = x_input[:, pair_list[j][0]], rep = depth)
        result_1 = QulacsSimulator().simulate(qc_cirq)
        #Second half of the circuit
        qcd_cirq = cirq.Circuit()
        IQP_circuit(qci = qcd_cirq, bn = bn, input_data = x_input[:, pair_list[j][1]], rep = depth)
        result_2 = QulacsSimulator().simulate(qcd_cirq)
        #Calculations for Gram matrix using inner product
        state_1 = result_1._final_simulator_state.reshape(1,-1)
        state_2 = result_2._final_simulator_state.reshape(1,-1)
        result_kernel = abs((np.conjugate(state_1) @ state_2.T)[0][0])**2
        result_gram_mat[pair_list[j][0], pair_list[j][1]] = result_kernel
        
    kernel_gram_mat = result_gram_mat + result_gram_mat.T - np.diag(result_gram_mat.diagonal())
    return kernel_gram_mat

In [ ]:
qr = QuantumRegister(1)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
miti_err = execute(meas_calibs, BasicAer.get_backend('qasm_simulator'), shots=1024, seed=8)
miti_err.result()

TypeError: ignored

In [ ]:
"""quantum kernel using Qiskit with error mitigation"""
#linear connection
def quantum_kernel_qiskit_IBMQ_sep_mitigated(bn, n_samples, data_x, depth):
    #Preparation for error mitigation
    qr = QuantumRegister(bn)
    meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
    miti_r = execute(meas_calibs,backend_sim)
    # miti_r = execute(meas_calibs, backend=backend_sim, initial_layout=[0,1,4,7],optimization_level = 3, shots=8192)
    cal_results = miti_r.result()
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
    meas_filter = meas_fitter.filter
    
    result_gram_mat = np.eye(n_samples)
    pair_list = []
    for pair in itertools.combinations(list(range(n_samples)), 2):
        pair_list.append(list(pair))
    
    #job list
    job_all = []
    
    #scaling and standardization
    x_standardize = preprocessing.scale(data_x)
    x_input = np.array([2*x_standardize for i in range(bn)])
    
    #calculations for Gram matrix
    for j in range(len(pair_list)):
        #quantum bits
        q = QuantumRegister(bn, "q")
        c = ClassicalRegister(bn,"c")

        #IQP circuit
        def IQP_circuit(qci, bn, input_data, rep):
            for _ in range(rep):
                for i in range(bn):
                    qci.h(q[i])
                for i in range(bn):
                    qci.u1(math.pi*func_one(input_data[i]), q[i])
                for i in range(bn-1):
                    qci.cu1(math.pi*func_two(input_data[i], input_data[i+1]), q[i], q[i+1])

        #First half of the circuit
        qc = QuantumCircuit(q, c)
        IQP_circuit(qci = qc, bn = bn, input_data = x_input[:, pair_list[j][0]], rep = depth)
        #Second half of the circuit
        qcd = QuantumCircuit(q, c)
        IQP_circuit(qci = qcd, bn = bn, input_data = x_input[:, pair_list[j][1]], rep = depth)
        qcd = qcd
        qcd_inverse = qcd.inverse()

        #Combine the circuits and perform the inversion test
        qc = qc.compose(qcd_inverse)
        for i in range(bn):
            qc.measure(q[bn-1-i], c[i])
        job_all.append(qc)
        
    #Number of circuits per job
    sep_num = 300
    def split_list(l, n):
        for idx in range(0, len(l), n):
            yield l[idx:idx + n]
    
    job_sep_list = list(split_list(job_all, sep_num))
    rc_list = []
    #Perform calculations for each job
    #Set the number of qubits to use and the number of shots
    for i in range(len(job_sep_list)):
        r = execute(job_sep_list[i], backend_sim,initial_layout=[0,1,4,7],optimization_level=3,shots=8192).result()
        mitigated_r = meas_filter.apply(r)
        for k in range(len(job_sep_list[i])):
            r_count = mitigated_r.get_counts(k)
            rc_list.append(r_count)
        
    for j in range(len(pair_list)):
        rc = rc_list[j]
        rc.setdefault('0'*bn, 0)
        #Estimate quantum kernel
        result_kernel = rc["0"*bn]/sum(rc.values())
        result_gram_mat[pair_list[j][0], pair_list[j][1]] = result_kernel
        
    kernel_gram_mat = result_gram_mat + result_gram_mat.T - np.diag(result_gram_mat.diagonal())
    return kernel_gram_mat

The following three cells are the source code for qLiNGAM, created by modifying the source code at the URL:<br>
https://github.com/cdt15/lingam <br>
The license for the above is as follows. <br>


Copyright (c) 2019 T.Ikeuchi, G.Haraoka, M.Ide, W.Kurebayashi, S.Shimizu

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
import numbers
from sklearn.utils import check_array, resample


class BootstrapMixin():
    def bootstrap(self, X, n_sampling):
        # Check parameters
        X = check_array(X)

        if isinstance(n_sampling, (numbers.Integral, np.integer)):
            if not 0 < n_sampling:
                raise ValueError(
                    'n_sampling must be an integer greater than 0.')
        else:
            raise ValueError('n_sampling must be an integer greater than 0.')

        # Bootstrapping
        adjacency_matrices = []
        for _ in range(n_sampling):
            model = self.fit(resample(X))
            adjacency_matrices.append(model.adjacency_matrix_)
        return BootstrapResult(adjacency_matrices)


class BootstrapResult(object):
    def __init__(self, adjacency_matrices):
        self._adjacency_matrices = adjacency_matrices

    @property
    def adjacency_matrices_(self):
        
        return self._adjacency_matrices

    def get_causal_direction_counts(self, n_directions=None, min_causal_effect=None, split_by_causal_effect_sign=False):
        # Check parameters
        if isinstance(n_directions, (numbers.Integral, np.integer)):
            if not 0 < n_directions:
                raise ValueError(
                    'n_directions must be an integer greater than 0')
        elif n_directions is None:
            pass
        else:
            raise ValueError('n_directions must be an integer greater than 0')

        if min_causal_effect is None:
            min_causal_effect = 0.0
        else:
            if not 0.0 < min_causal_effect:
                raise ValueError(
                    'min_causal_effect must be an value greater than 0.')

        # Count causal directions
        directions = []
        for am in self._adjacency_matrices:
            direction = np.array(np.where(np.abs(am) > min_causal_effect))
            if split_by_causal_effect_sign:
                signs = np.array([np.sign(am[i][j])
                                  for i, j in direction.T]).astype('int64').T
                direction = np.vstack([direction, signs])
            directions.append(direction.T)
        directions = np.concatenate(directions)

        if len(directions) == 0:
            cdc = {'from': [], 'to': [], 'count': []}
            if split_by_causal_effect_sign:
                cdc['sign'] = []
            return cdc

        directions, counts = np.unique(directions, axis=0, return_counts=True)
        sort_order = np.argsort(-counts)
        sort_order = sort_order[:n_directions] if n_directions is not None else sort_order
        counts = counts[sort_order]
        directions = directions[sort_order]

        cdc = {
            'from': directions[:, 1].tolist(),
            'to': directions[:, 0].tolist(),
            'count': counts.tolist()
        }
        if split_by_causal_effect_sign:
            cdc['sign'] = directions[:, 2].tolist()

        return cdc

    def get_directed_acyclic_graph_counts(self, n_dags=None, min_causal_effect=None, split_by_causal_effect_sign=False):
        # Check parameters
        if isinstance(n_dags, (numbers.Integral, np.integer)):
            if not 0 < n_dags:
                raise ValueError('n_dags must be an integer greater than 0')
        elif n_dags is None:
            pass
        else:
            raise ValueError('n_dags must be an integer greater than 0')

        if min_causal_effect is None:
            min_causal_effect = 0.0
        else:
            if not 0.0 < min_causal_effect:
                raise ValueError(
                    'min_causal_effect must be an value greater than 0.')

        # Count directed acyclic graphs
        dags = []
        for am in self._adjacency_matrices:
            dag = np.abs(am) > min_causal_effect
            if split_by_causal_effect_sign:
                direction = np.array(np.where(dag))
                signs = np.zeros_like(dag).astype('int64')
                for i, j in direction.T:
                    signs[i][j] = np.sign(am[i][j]).astype('int64')
                dag = signs
            dags.append(dag)

        dags, counts = np.unique(dags, axis=0, return_counts=True)
        sort_order = np.argsort(-counts)
        sort_order = sort_order[:n_dags] if n_dags is not None else sort_order
        counts = counts[sort_order]
        dags = dags[sort_order]

        if split_by_causal_effect_sign:
            dags = [{
                'from': np.where(dag)[1].tolist(),
                'to': np.where(dag)[0].tolist(),
                'sign': [dag[i][j] for i, j in np.array(np.where(dag)).T]} for dag in dags]
        else:
            dags = [{
                'from': np.where(dag)[1].tolist(),
                'to': np.where(dag)[0].tolist()} for dag in dags]

        return {
            'dag': dags,
            'count': counts.tolist()
        }

    def get_probabilities(self, min_causal_effect=None):
        # check parameters
        if min_causal_effect is None:
            min_causal_effect = 0.0
        else:
            if not 0.0 < min_causal_effect:
                raise ValueError(
                    'min_causal_effect must be an value greater than 0.')

        shape = self._adjacency_matrices[0].shape
        bp = np.zeros(shape)
        for B in self._adjacency_matrices:
            bp += np.where(np.abs(B) > min_causal_effect, 1, 0)
        bp = bp/len(self._adjacency_matrices)

        if int(shape[1]/shape[0]) == 1:
            return bp
        else:
            return np.hsplit(bp, int(shape[1]/shape[0]))


class LongitudinalBootstrapResult(object):
    
    def __init__(self, adjacency_matrices, n_timepoints):
        
        self._adjacency_matrices = adjacency_matrices
        self._n_timepoints = n_timepoints

    @property
    def adjacency_matrices_(self):
        
        return self._adjacency_matrices

    def get_causal_direction_counts(self, n_directions=None, min_causal_effect=None, split_by_causal_effect_sign=False):
        
        # Check parameters
        if isinstance(n_directions, (numbers.Integral, np.integer)):
            if not 0 < n_directions:
                raise ValueError(
                    'n_directions must be an integer greater than 0')
        elif n_directions is None:
            pass
        else:
            raise ValueError('n_directions must be an integer greater than 0')

        if min_causal_effect is None:
            min_causal_effect = 0.0
        else:
            if not 0.0 < min_causal_effect:
                raise ValueError(
                    'min_causal_effect must be an value greater than 0.')

        # Count causal directions
        cdc_list = []
        for t in range(self._n_timepoints):

            directions = []
            for m in self._adjacency_matrices:
                am = np.concatenate([*m[t]], axis=1)
                direction = np.array(np.where(np.abs(am) > min_causal_effect))
                if split_by_causal_effect_sign:
                    signs = np.array([np.sign(am[i][j])
                                      for i, j in direction.T]).astype('int64').T
                    direction = np.vstack([direction, signs])
                directions.append(direction.T)
            directions = np.concatenate(directions)

            if len(directions) == 0:
                cdc = {'from': [], 'to': [], 'count': []}
                if split_by_causal_effect_sign:
                    cdc['sign'] = []
                cdc_list.append(cdc)
                continue

            directions, counts = np.unique(
                directions, axis=0, return_counts=True)
            sort_order = np.argsort(-counts)
            sort_order = sort_order[:n_directions] if n_directions is not None else sort_order
            counts = counts[sort_order]
            directions = directions[sort_order]

            cdc = {
                'from': directions[:, 1].tolist(),
                'to': directions[:, 0].tolist(),
                'count': counts.tolist()
            }
            if split_by_causal_effect_sign:
                cdc['sign'] = directions[:, 2].tolist()

            cdc_list.append(cdc)

        return cdc_list

    def get_directed_acyclic_graph_counts(self, n_dags=None, min_causal_effect=None, split_by_causal_effect_sign=False):
        
        # Check parameters
        if isinstance(n_dags, (numbers.Integral, np.integer)):
            if not 0 < n_dags:
                raise ValueError('n_dags must be an integer greater than 0')
        elif n_dags is None:
            pass
        else:
            raise ValueError('n_dags must be an integer greater than 0')

        if min_causal_effect is None:
            min_causal_effect = 0.0
        else:
            if not 0.0 < min_causal_effect:
                raise ValueError(
                    'min_causal_effect must be an value greater than 0.')

        # Count directed acyclic graphs
        dagc_list = []
        for t in range(self._n_timepoints):

            dags = []
            for m in self._adjacency_matrices:
                am = np.concatenate([*m[t]], axis=1)

                dag = np.abs(am) > min_causal_effect
                if split_by_causal_effect_sign:
                    direction = np.array(np.where(dag))
                    signs = np.zeros_like(dag).astype('int64')
                    for i, j in direction.T:
                        signs[i][j] = np.sign(am[i][j]).astype('int64')
                    dag = signs
                dags.append(dag)

            dags, counts = np.unique(dags, axis=0, return_counts=True)
            sort_order = np.argsort(-counts)
            sort_order = sort_order[:n_dags] if n_dags is not None else sort_order
            counts = counts[sort_order]
            dags = dags[sort_order]

            if split_by_causal_effect_sign:
                dags = [{
                    'from': np.where(dag)[1].tolist(),
                    'to': np.where(dag)[0].tolist(),
                    'sign': [dag[i][j] for i, j in np.array(np.where(dag)).T]} for dag in dags]
            else:
                dags = [{
                    'from': np.where(dag)[1].tolist(),
                    'to': np.where(dag)[0].tolist()} for dag in dags]

            dagc_list.append({
                'dag': dags,
                'count': counts.tolist()
            })

        return dagc_list

    def get_probabilities(self, min_causal_effect=None):
        
        # check parameters
        if min_causal_effect is None:
            min_causal_effect = 0.0
        else:
            if not 0.0 < min_causal_effect:
                raise ValueError(
                    'min_causal_effect must be an value greater than 0.')

        prob = np.zeros(self._adjacency_matrices[0].shape)
        for adj_mat in self._adjacency_matrices:
            prob += np.where(np.abs(adj_mat) > min_causal_effect, 1, 0)
        prob = prob/len(self._adjacency_matrices)

        return prob

In [ ]:
from abc import ABCMeta, abstractmethod
from sklearn.linear_model import LassoLarsIC, LinearRegression
from sklearn.utils import check_array

class _BaseLiNGAM(BootstrapMixin, metaclass=ABCMeta):

    def __init__(self, random_state=None):
        
        self._random_state = random_state
        self._causal_order = None
        self._adjacency_matrix = None

    @abstractmethod
    def fit(self, X):
        """Subclasses should implement this method!
        Fit the model to X.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        Returns
        -------
        self : object
            Returns the instance itself.
        """

    def estimate_total_effect(self, X, from_index, to_index):
        # Check parameters
        X = check_array(X)

        # from_index + parents indices
        parents = np.where(np.abs(self.adjacency_matrix_[from_index]) > 0)[0].tolist()
        predictors = [from_index]
        predictors.extend(parents)

        # estimate total effect by Adaptive Lasso
        coef = self._predict_adaptive_lasso(X, predictors, to_index)
        return coef[0]

    def _predict_adaptive_lasso(self, X, predictors, target, gamma=1.0):
        
        lr = LinearRegression()
        lr.fit(X[:, predictors], X[:, target])
        weight = np.power(np.abs(lr.coef_), gamma)
        reg = LassoLarsIC(criterion='bic')
        reg.fit(X[:, predictors] * weight, X[:, target])
        return reg.coef_ * weight

    def _estimate_adjacency_matrix(self, X):
        
        B = np.zeros([X.shape[1], X.shape[1]], dtype='float64')
        for i in range(1, len(self._causal_order)):
            coef = self._predict_adaptive_lasso(
                X, self._causal_order[:i], self._causal_order[i])
            B[self._causal_order[i], self._causal_order[:i]] = coef

        self._adjacency_matrix = B
        return self

    @property
    def causal_order_(self):
        
        return self._causal_order

    @property
    def adjacency_matrix_(self):
        
        return self._adjacency_matrix

In [ ]:
from sklearn.utils import check_array
from sklearn.preprocessing import scale
"""
"_mutual_information" used in the Tkernel calculation should be selected 
according to the quantum kernel calculation method to use
"""
class qLiNGAM(_BaseLiNGAM):

    def __init__(self, random_state=None, prior_knowledge=None, measure='pwling'):
        
        super().__init__(random_state)
        self._prior_knowledge = prior_knowledge
        self._measure = measure

    def fit(self, X):
        
        # Check parameters
        X = check_array(X)
        # number of variables
        n_features = X.shape[1]

        if self._prior_knowledge is not None:
            self._Aknw = check_array(self._prior_knowledge)
            self._Aknw = np.where(self._Aknw < 0, np.nan, self._Aknw)
            if (n_features, n_features) != self._Aknw.shape:
                raise ValueError('The shape of prior knowledge must be (n_features, n_features)')
        else:
            self._Aknw = None

        # Causal discovery
        U = np.arange(n_features)
        K = []
        X_ = np.copy(X)
        if self._measure == 'kernel':
            X_ = scale(X_)

        for _ in range(n_features):
            if self._measure == 'kernel':
                m = self._search_causal_order_kernel(X_, U)
            else:
                m = self._search_causal_order(X_, U)
            for i in U:
                if i != m:
                    X_[:, i] = self._residual(X_[:, i], X_[:, m])
            K.append(m)
            U = U[U != m]

        self._causal_order = K
        return self._estimate_adjacency_matrix(X)

    def _residual(self, xi, xj):
        """The residual when xi is regressed on xj."""
        return xi - (np.cov(xi, xj)[0, 1] / np.var(xj)) * xj

    def _search_candidate(self, U):
        """ Search for candidate features """
        # If no prior knowledge is specified, nothing to do.
        if self._Aknw is None:
            return U, []

        # Find exogenous features
        Uc = []
        for j in U:
            index = U[U != j]
            if self._Aknw[j][index].sum() == 0:
                Uc.append(j)

        # Find endogenous features, and then find candidate features
        if len(Uc) == 0:
            U_end = []
            for j in U:
                index = U[U != j]
                if np.nansum(self._Aknw[j][index]) > 0:
                    U_end.append(j)

            # Find sink features (original)
            for i in U:
                index = U[U != i]
                if self._Aknw[index, i].sum() == 0:
                    U_end.append(i)
            Uc = [i for i in U if i not in set(U_end)]

        # make V^(j)
        Vj = []
        for i in U:
            if i in Uc:
                continue
            if self._Aknw[i][Uc].sum() == 0:
                Vj.append(i)
        return Uc, Vj
    
    def _mutual_information_quantum_cirq_INOCCO(self, x1, x2, param):
        """Calculate the mutual informations with INOCCO for quantum computing"""
        
        eps = 1e-6
        n = len(x1)
        K1 = quantum_kernel_cirq_ring(bn=5, n_samples=n, data_x=x1, depth=2)
        
        transformer = KernelCenterer().fit(K1)
        K1_centered = transformer.transform(K1)
        
        K2 = quantum_kernel_cirq_ring(bn=5, n_samples=n, data_x=x2, depth=2)
        
        transformer = KernelCenterer().fit(K2)
        K2_centered = transformer.transform(K2)
        
        K1_temp = K1_centered+n*eps*np.identity(n)
        K2_temp = K2_centered+n*eps*np.identity(n)
        
        K1_r = K1_centered @ np.linalg.inv(K1_temp)
        K2_r = K2_centered @ np.linalg.inv(K2_temp)
        
        return np.trace(K2_r @ K1_r)
    
    def _mutual_information_quantum_cirq_INOCCO_linear(self, x1, x2, param):
        eps = 1e-6
        n = len(x1)
        K1 = quantum_kernel_cirq_linear(bn=4, n_samples=n, data_x=x1, depth=1)
        
        transformer = KernelCenterer().fit(K1)
        K1_centered = transformer.transform(K1)
        
        K2 = quantum_kernel_cirq_linear(bn=4, n_samples=n, data_x=x2, depth=1)
        
        transformer = KernelCenterer().fit(K2)
        K2_centered = transformer.transform(K2)
        
        K1_temp = K1_centered+n*eps*np.identity(n)
        K2_temp = K2_centered+n*eps*np.identity(n)
        
        K1_r = K1_centered @ np.linalg.inv(K1_temp)
        K2_r = K2_centered @ np.linalg.inv(K2_temp)
        
        return np.trace(K2_r @ K1_r)
    
    def _mutual_information_quantum_IBMQ_mitigated(self, x1, x2, param):
        eps = 1e-6
        n = len(x1)
        K1 = quantum_kernel_qiskit_IBMQ_sep_mitigated(bn=4, n_samples=n, data_x=x1, depth=1)
        
        transformer = KernelCenterer().fit(K1)
        K1_centered = transformer.transform(K1)
        
        K2 = quantum_kernel_qiskit_IBMQ_sep_mitigated(bn=4, n_samples=n, data_x=x2, depth=1)
        
        transformer = KernelCenterer().fit(K2)
        K2_centered = transformer.transform(K2)
        
        K1_temp = K1_centered+n*eps*np.identity(n)
        K2_temp = K2_centered+n*eps*np.identity(n)
        
        K1_r = K1_centered @ np.linalg.inv(K1_temp)
        K2_r = K2_centered @ np.linalg.inv(K2_temp)
        
        return np.trace(K2_r @ K1_r)
    
    def _search_causal_order_kernel(self, X, U):
        """Search the causal ordering by kernel method."""
        Uc, Vj = self._search_candidate(U)
        if len(Uc) == 1:
            return Uc[0]

        if X.shape[0] > 1000:
            param = [2e-3, 0.5]
        else:
            param = [2e-2, 1.0]

        Tkernels = []
        for j in Uc:
            Tkernel = 0
            for i in U:
                if i != j:
                    ri_j = X[:, i] if j in Vj and i in Uc else self._residual(
                        X[:, i], X[:, j])
                    Tkernel += self._mutual_information_quantum_IBMQ_mitigated(X[:, j], ri_j, param)
            Tkernels.append(Tkernel)

        return Uc[np.argmin(Tkernels)]

# Data Preparation: Obtain real-world medical data
UCI Heart Disease Data Set is available at the following URL (Data used is "processed.cleveland.data"): <br>
https://archive.ics.uci.edu/ml/datasets/Heart+Disease <br>
Pima Indians Diabetes Database is available at the following URL:<br>
https://www.kaggle.com/uciml/pima-indians-diabetes-database

In [ ]:
"""UCI Heart Disease Data Set"""
#Get the data from the above URL and save it as heartdata
heartdata = pd.DataFrame(pd.read_fwf(r'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data', header=None))
heartdata = heartdata[0].str.split(',', expand=True)
heartdata_full = heartdata[[0,1,8]] # ['age','cp','exang']
heartdata_short = heartdata_full.sample(n=100, random_state=0) #short version
heartdata_short.head()

,0,1,8
225,34.0,0.0,0.0
152,67.0,0.0,0.0
228,54.0,1.0,1.0
201,64.0,0.0,1.0
52,44.0,1.0,0.0


In [ ]:
# """Pima Indians Diabetes Database"""
# #Get the data from the above URL and save it as diabetesdata
# diabetesdata = pd.read_csv(r'https://www.kaggle.com/uciml/pima-indians-diabetes-database/diabetes.csv')
# diabetesdata[['BloodPressure','Glucose','Insulin','BMI']] = diabetesdata[['BloodPressure','Glucose','Insulin','BMI']].replace(0, np.NaN)
# diabetesdata_new=diabetesdata.dropna()
# diabetesdata_full = diabetesdata_new[['Age','Insulin','Glucose']] #full version
# diabetesdata_short = diabetesdata_full.sample(n=100, random_state=0) #short version
# diabetesdata_short

# Preliminary experiment

In [ ]:
index = 1 #Number of experiments
model_mass = np.empty([index, 3, 3])
model_result = list() #list to put correct or incorrect answers
for i in range(index):
    np.random.seed(i)
    n = 100
    e = lambda n: np.random.laplace(0, 1, n)
    x0 = e(n)
    x1 = 0.3*x0 + e(n)
    x2 = 0.3*x1 + 0.3*x0 + e(n)
    X = pd.DataFrame(np.array([x0, x1, x2]).T ,columns=['x0', 'x1', 'x2'])
    model_quantum = qLiNGAM(measure='kernel')
    model_quantum.fit(X)
    model_mass[i] = model_quantum.adjacency_matrix_
    model_result.append(1) if np.sum(np.ceil(np.tril(model_quantum.adjacency_matrix_)))==3 else model_result.append(0)
print(sum(model_result))
print(model_result)

AttributeError: ignored

# Experiments with real-world medical data: Part 1

In [ ]:
"""Need to change qLiNGAM according to the quantum kernel to be implemented"""
model_quantum = qLiNGAM(measure='kernel')
model_quantum.fit(heartdata_short)
print(model_quantum.adjacency_matrix_)
print(heartdata_short.columns)
make_dot(model_quantum.adjacency_matrix_)

# Experiments with real-world medical data: Part 2

In [ ]:
# """Need to change qLiNGAM according to the quantum kernel to be implemented"""
# model_quantum = qLiNGAM(measure='kernel')
# model_quantum.fit(diabetesdata_short)
# print(model_quantum.adjacency_matrix_)
# print(diabetesdata_short.columns)
# make_dot(model_quantum.adjacency_matrix_)